 
#### Rag Evaluation: Retriever Evaluation
- ปรับค่า k ในการดึงเอกสาร
- เปลี่ยน embedding model
- ปรับวิธีการแบ่ง chunk ของเอกสาร
- เพิ่มการกรองเอกสารที่ไม่เกี่ยวข้อง

In [ ]:
from typing import List, Dict, Any, Optional
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from tqdm import tqdm

def evaluate_retriever_precision_recall(
    vectorstore: Any,
    test_data: List[Dict[str, Any]],
    k_value: int = 5,
    similarity_threshold: float = 0.3,
    embedding_model: str = "text-embedding-3-small",
    batch_size: int = 10,
    openai_client: Optional[OpenAI] = None
) -> Dict[str, Any]:
    """
    Evaluate precision and recall of a retriever using OpenAI embeddings.
    
    Parameters:
    -----------
    vectorstore: Any
        The vector store to evaluate
    test_data: List[Dict[str, Any]]
        List of test cases with "input" (query) and "expected_output" (expected relevant content)
    k_value: int
        Number of documents to retrieve for each query
    similarity_threshold: float
        Threshold for considering a document relevant
    embedding_model: str
        OpenAI embedding model to use
    batch_size: int
        Batch size for embedding requests to reduce API calls
    openai_client: Optional[OpenAI]
        Pre-initialized OpenAI client (will create one if None)
        
    Returns:
    --------
    Dict[str, Any]
        Evaluation results including per-query metrics and averages
    """
    
    # Create or use provided OpenAI client
    client = openai_client or OpenAI()
    
    results = []
    
    # Process test data in batches to reduce API calls
    for i in tqdm(range(0, len(test_data), batch_size), desc="Evaluating batches"):
        batch = test_data[i:i+batch_size]
        
        # Process each item in the batch
        batch_questions = []
        batch_expected_outputs = []
        batch_retrieved_docs = []
        
        for item in batch:
            question = item["question"]
            expected_output = item["ideal_context"][0]
            batch_questions.append(question)
            batch_expected_outputs.append(expected_output)
            
            # Retrieve documents
            retrieved_docs = vectorstore.similarity_search(question, k=k_value)
            retrieved_contents = [doc.page_content for doc in retrieved_docs]
            batch_retrieved_docs.append(retrieved_contents)
        
        # Flatten for embedding
        all_expected_outputs = batch_expected_outputs
        all_retrieved_contents = [content for contents in batch_retrieved_docs for content in contents]
        
        # Get embeddings in batches
        expected_embedding_response = client.embeddings.create(
            model=embedding_model,
            input=all_expected_outputs
        )
        expected_embeddings = [item.embedding for item in expected_embedding_response.data]
        
        retrieved_embedding_response = client.embeddings.create(
            model=embedding_model,
            input=all_retrieved_contents
        )
        retrieved_embeddings = [item.embedding for item in retrieved_embedding_response.data]
        
        # Process results for each item in the batch
        retrieved_idx = 0
        for i, (question, expected_output) in enumerate(zip(batch_questions, batch_expected_outputs)):
            expected_embedding = expected_embeddings[i]
            item_retrieved_count = len(batch_retrieved_docs[i])
            
            # Calculate similarities for this item's retrieved docs
            similarities = []
            for j in range(item_retrieved_count):
                retrieved_embedding = retrieved_embeddings[retrieved_idx]
                retrieved_idx += 1
                
                # Convert to numpy arrays
                expected_array = np.array(expected_embedding).reshape(1, -1)
                retrieved_array = np.array(retrieved_embedding).reshape(1, -1)
                similarity = cosine_similarity(expected_array, retrieved_array)[0][0]
                similarities.append(float(similarity))  # Convert numpy types to native Python types
            
            # Calculate metrics
            relevant_docs = [sim for sim in similarities if sim > similarity_threshold]
            precision = len(relevant_docs) / item_retrieved_count if item_retrieved_count else 0
            recall = 1.0 if relevant_docs else 0.0  # Assuming one relevant document
            f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
            
            results.append({
                "question": question,
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score,
                "similarities": similarities,
                "max_similarity": max(similarities) if similarities else 0,
                "relevant_count": len(relevant_docs),
                "retrieved_count": item_retrieved_count,
                "expected_output": expected_output,
                "retrieved_docs": retrieved_docs
            })
    
    # Calculate averages
    avg_precision = sum(r["precision"] for r in results) / len(results)
    avg_recall = sum(r["recall"] for r in results) / len(results)
    avg_f1 = sum(r["f1_score"] for r in results) / len(results)
    avg_max_similarity = sum(r["max_similarity"] for r in results) / len(results)
    
    return {
        "results": results,
        "average_precision": avg_precision,
        "average_recall": avg_recall,
        "average_f1": avg_f1,
        "average_max_similarity": avg_max_similarity,
        "total_queries": len(results),
        "threshold_used": similarity_threshold
    }

In [62]:
from deepeval.metrics import ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from typing import Any, List, Dict

def evaluate_context_relevancy(
    vectorstore: Any,
    test_data: List[Dict[str, Any]],
    k_value: int = 5
) -> Dict[str, Any]:
    """ประเมินความเกี่ยวข้องของบริบทที่ดึงมา"""
    
    metric = ContextualRelevancyMetric()
    results = []
    
    for item in test_data:
        question = item["question"]
        
        # ดึงเอกสารจาก vectorstore
        retrieved_docs = vectorstore.similarity_search(question, k=k_value)
        
        # สร้าง test case สำหรับ DeepEval
        # Note: actual_output is required but not used for this metric
        test_case = LLMTestCase(
            input=question,
            actual_output="",  # Not used for this metric but required
            retrieval_context=[doc.page_content for doc in retrieved_docs]
        )
        
        # ประเมินด้วย DeepEval
        score = metric.measure(test_case)
        
        results.append({
            "question": question,
            "retrieved_docs": retrieved_docs,
            "context_relevancy_score": score
        })
    
    # คำนวณค่าเฉลี่ย
    avg_score = sum(r["context_relevancy_score"] for r in results) / len(results)
    
    return {
        "results": results,
        "average_context_relevancy": avg_score
    }

In [63]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from src.utils.app_config import AppConfig
from src.components.ragchain_runner import RAGChainRunner
import os

# โหลด config
config = AppConfig.from_files(model_config_path = "configs/model_config.yaml", environment_config_path = "config.yaml")
 
os.environ["OPENAI_API_KEY"] = config.openai_token


test_data_path = "data/evaluation/golden_dataset_v1.json"
# โหลด test data
with open(test_data_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

# สร้าง RAG chain
rag_runner = RAGChainRunner(config)

# ประเมิน Retriever

# retriever_precision_recall = evaluate_retriever_precision_recall(
#     vectorstore=rag_runner.vectorstore,
#     test_data=test_data,
#     k_value= 4
# )

context_relevancy = evaluate_context_relevancy(
    vectorstore=rag_runner.vectorstore,
    test_data=test_data,
    k_value=4
)

/home/godoftan/dev/typhoon/.venv/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [64]:
context_relevancy

{'results': [{'question': 'จุดเด่นหลักของ Quant Offside 3X คืออะไร?',
   'retrieved_docs': [Document(id='19917f76-0aef-46d9-8ffa-722ead786bee', metadata={'source': 'data/raw/overall.txt'}, page_content='**Introduction to Quant Offside 3X**\n\nสัมมนา Quant Trading ที่ใหญ่ที่สุดของปี พร้อมพลิกโลกการลงทุนของคุณให้เหนือชั้นกว่าเดิมด้วย Data-Driven Trading และ AI Investing! **Quant Offside 3X rerun เหมาะกับใคร?**\n\n- ผมยังไม่รู้จะไปทางไหน? - คอร์สนี้จะทำให้คุณเห็นภาพกว้างมากขึ้น\n- คุณจะได้เห็นว่าเหล่า Trader ที่ทำกำไร หรือ CEO , CFO ที่เค้าทำงานด้านการลงทุน Hedge Fund เค้ามีมุมมอง มีจุดเริ่มต้นอย่างไร จนมาถึงวันที่เค้าทำกำไรได้ และเริ่มอยู่ตัวในโลก Quant เพื่อการลงทุน\n- อยากทำระบบเทรด Robot Trade แต่ไม่มี Idea\n- คุณจะได้ Idea มากมายที่ Speaker แต่ละท่านใช้\n- และ speaker แต่ละคนไม่ว่าจะเป็นสายเทรดมือ ทำระบบ Trade เป็น ผู้จัดการกองทุน เหล่า CEO, CFO บอกเลยว่าไม่ธรรมดาครับ และพวกเราคุยกันด้วย Data , Quant, Math แน่นอนจะทำให้คุณเห็นภาพมากขึ้น และเอาไปประยุกต์ใช้กับระบบ Trade หรือ Robot Tra